In [ ]:
# this script parses important insurer information from all xls and xlsx files
# from 2014 and 2015! finally!
# original data can be found here: https://www.cms.gov/apps/mlr/mlr-search.aspx

# author: Grace Guan
# date created: 12/6/17

import pandas as pd
import numpy as np
import xlrd
import glob
import os
import time

In [ ]:
def parse_input(df14, df15, xls):
    
    sheet0 = xls.parse(0)
    
    # 2014, 2015, and 2016 format
    if sheet0.iloc[2,0] == 'Company Name:': # should say "Company Name:"
        # hios id is 11C (9,1) in sheet 0
        # company name 4C (2,1) is in sheet 0
        # company state is 12C (10,1) in sheet 0
        # print(sheet0.iloc[9,1]) hios id
        # print(sheet0.iloc[2,1]) company name
        # print(sheet0.iloc[10,1]) company state 
        # print(sheet0.iloc[16,1]) reporting year
        reporting_year = sheet0.iloc[16,1]
        if reporting_year == 'No':
            reporting_year = sheet0.iloc[17,1]
        
        # individual member months is in section 1.7 59E (57,3) in sheet 1
        # small group member months is in section 1.7 59K (57,9) in sheet 1
        
        sheet1 = xls.parse(1)
        # print(sheet1.iloc[57,3]) individual member months
        # print(sheet1.iloc[57,9]) small group member months

        # reinsurance is in section 1.9 which is 15E in sheet 2
        # individual risk adjustment is in section 1.10 which is 16E in sheet 2
        # small group risk adjustment is in section 1.10 which is 16K in sheet 2
        sheet2 = xls.parse(2)
        # print(sheet2.iloc[13,3]) reinsurance
        # print(sheet2.iloc[14,3]) individual risk adjustment
        # print(sheet2.iloc[14,9]) small group risk adjustment

        if reporting_year == '2014':
            df14.loc[df14.shape[0]] = [sheet0.iloc[9,1], # hios id
                                   sheet0.iloc[2,1],  # company name
                                   sheet0.iloc[10,1], # company state
                                   sheet1.iloc[57,3], # individual member months
                                   sheet1.iloc[57,9], # small group member months
                                   sheet2.iloc[13,3], # reinsurance
                                   sheet2.iloc[14,3], # individual risk adjustment
                                   sheet2.iloc[14,9]] # small group risk adjustment
        elif reporting_year == '2015':
            df15.loc[df15.shape[0]] = [sheet0.iloc[9,1], # hios id
                                   sheet0.iloc[2,1],  # company name
                                   sheet0.iloc[10,1], # company state
                                   sheet1.iloc[57,3], # individual member months
                                   sheet1.iloc[57,9], # small group member months
                                   sheet2.iloc[13,3], # reinsurance
                                   sheet2.iloc[14,3], # individual risk adjustment
                                   sheet2.iloc[14,9]] # small group risk adjustment
    # else: # 2011, 2012, 2013 format
        #print(sheet0.iloc[9,3]) # hios id
        #print(sheet0.iloc[4,2]) # company name
        #print(sheet0.iloc[8,7]) # company state
        #print(sheet0.iloc[79,8]) # individual member months
        #print(sheet0.iloc[79,13]) # small group member months
        # reinsurance, risk adjustment missing?
    
    #print("parse_input defined")

In [ ]:
# create the dataframe

t0 = time.time()

errorcount = 0

mydf14 = pd.DataFrame(columns=('HIOS_ID', 'COMPANY_NAME', 'STATE', 
                        'IND_MEMBER_MONTHS', 'SG_MEMBER_MONTHS',
                       'REINSURANCE', 'IND_RISK_ADJ', 'SG_RISK_ADJ'))
mydf15 = pd.DataFrame(columns=('HIOS_ID', 'COMPANY_NAME', 'STATE', 
                        'IND_MEMBER_MONTHS', 'SG_MEMBER_MONTHS',
                       'REINSURANCE', 'IND_RISK_ADJ', 'SG_RISK_ADJ'))

print(os.getcwd())
os.chdir('C:/Users/guanz/Desktop/new_input_2')
FileList = glob.glob('*.xls*')

FailedFileList = list()

for File in FileList:
    try:
        myxls = pd.ExcelFile(File)
        print(File + " success")
        parse_input(mydf14, mydf15, myxls)
    except xlrd.XLRDError:
        print(File + " failure")
        errorcount = errorcount + 1
        FailedFileList.append(File)
        pass


print("There were " + str(errorcount) + " encrypted files.")
print(mydf14)
print(mydf15)

t1 = time.time()

print(t1-t0)